# 1-Model

在当前人工智能领域, 主流大模型从架构上大致可分为稠密（Dense）模型和混合专家（Mixture of Expert, MoE）模型 。稠密模型中所有参数在每次计算时都会参与运算；混合专家模型则将不同的 “专家” 模块组合, 根据输入选择合适的专家处理, 能在保证效果的同时减少计算量和参数量.

MiniMind 系列模型在 Llama 3.1 的基础上设计, 基于经典的 Transformer Deocder-Only 架构，在这一板块, 我们将围绕 MiniMind 系列模型的源代码展开学习.

## MiniMind Dense Model

作者提供了对其 MiniMind Dense Model 模型结构的可视化：

![image](./images/LLM-structure.png)

In [1]:
import math
import struct
import inspect
import time

from model.LMConfig import LMConfig
from typing import Any, Optional, Tuple, List
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

### 均方根层归一化 (Root Mean Square Layer Normalization, RMSNorm)

RMSNorm 是对 LayerNorm 的一个改进,  没有做 re-center 操作（移除了均值项）, 可以看作 LayerNorm 在均值为零时的特例, 使用平方根均值归一化降低噪声影响。

- **Layer Norm**

$$y = \frac{x-E(x)}{\sqrt{Var(x) + \epsilon}} * \gamma + \beta$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), 层归一化对 embedding_dim 维度进行归一化操作, 其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢,  $\gamma$,  $\beta$ 均为可学习参数。

- **RMS Norm**

$$y=\frac{x}{\sqrt{\frac{1}{n}\sum^n_{i=1}x^2_i + \epsilon}} * \gamma$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), RMS Norm 对 embedding_dim 维度进行归一化,其中,  其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢, $\gamma$ 为可学习参数.

不难发现, 当均值为零时, Layer Norm 退化为 RMS Norm. 这是因为 RMS Norm 在 Layer Norm 的基础上舍弃了中心化操作, 仅用缩放进行归一化, 其不改变数据原本的分布, 有利于激活函数输出的稳定.

---

#### RMSNorm 相比 LayerNorm 的核心优势
##### 1. 计算更高效，推理/训练速度更快
- **减少计算量**：去掉了「均值计算」和「均值中心化」步骤，仅需计算均方根（RMS），浮点运算（FLOPs）减少约 15%-25%。
- **内存访问更友好**：无需存储和计算均值，减少了中间变量的读写，尤其在大模型（如 7B/13B 参数）中，推理延迟和训练耗时的优化更明显。
- 实际效果：在同等硬件下，RMSNorm 能让 Transformer 训练/推理速度提升 5%-15%，且不损失模型性能。

##### 2. 简化参数，降低过拟合风险
- LayerNorm 有两组可学习参数（$ \gamma, \beta $），RMSNorm 仅保留 $ \gamma $，参数总量减少约一半（归一化层维度通常等于模型隐层维度，如 768/4096，累计参数节省可观）。
- 更少的参数意味着模型更难「过拟合」训练数据，尤其在小样本训练或模型轻量化场景中，泛化能力更稳定。

##### 3. 缓解均值中心化的副作用，训练更稳定
LayerNorm 的「均值中心化」会强制让输入特征的均值为 0，但在 Transformer 的残差连接（Residual Connection）中，残差流的均值会被反复扰动，可能导致：
- 梯度传播不稳定（尤其是深层模型）；
- 激活值分布被过度约束，影响模型表达能力。

RMSNorm 去掉均值中心化，仅通过 RMS 缩放控制激活值的**尺度**，保留了特征的原始分布信息，让残差流更平滑，深层模型的训练稳定性显著提升（这也是 GPT-3、LLaMA 等大模型选择 RMSNorm 的关键原因）。

##### 4. 对异常值更鲁棒
RMS 基于「平方和的平均」计算，相比 LayerNorm 的「方差（基于均值偏差）」，对极端异常值的敏感度更低。在自然语言处理中，文本序列的 token 分布往往存在长尾特征，RMSNorm 能更稳定地约束激活值范围，避免异常值导致的训练震荡。

##### 5. 与 Transformer 架构的兼容性更好
Transformer 的核心是「自注意力+前馈网络+残差连接」，RMSNorm 的设计更贴合残差流的特性：
- 不破坏残差连接的恒等映射特性（均值中心化会轻微改变残差流的恒等性）；
- 与 SwiGLU、GELU 等现代激活函数配合更默契，能进一步提升模型收敛速度和最终效果。

---

In [2]:
class RMSNorm(torch.nn.Module):
    def __init__(self,  dim: int,  eps: float):
        super().__init__()
        self.eps = eps # 防止分母等于零导致结果上溢
        self.weight = nn.Parameter(torch.ones(dim)) # 可学习的缩放参数

    def forward(self,  x):
        return self.weight * (x.float() * torch.rsqrt(x.pow(2).mean(-1,  keepdim=True) + self.eps)).type_as(x)

### Rotary Position Embedding, RoPE

旋转位置编码是一种能将相对位置信息集成到 self-attention 中, 进而提升 transformer 架构性能的位置编码方式, 和绝对位置编码相比, RoPE 具有很好的外推性, 是目前的主流位置编码方式.

外推性的解释, 通俗来说就是训练的时候限制了 512 的上下文长度，那么推理时如果面对超过该长度的文本，LLM 可能无法正确处理.

- **绝对位置编码**

绝对位置编码是早期 Transformer 架构采用的绝对位置编码方案，及那个每个位置映射为固定的向量表示.

$$f_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i,i)=\boldsymbol{W}_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i+\boldsymbol{p}_i)$$

其中编码向量 $p_i$ 的计算使用如下公式：

$$\boldsymbol{p}_{i,2t}=\sin\left(k/10000^{2t/d}\right), \boldsymbol{p}_{i,2t+1}=\cos\left(k/10000^{2t/d}\right)$$

正如其名，绝对位置编码只考虑了输入序列中的绝对位置关系，对于 token 之间的相对信息则没有纳入考虑.

- **旋转位置编码**

假定 query 和 key 的内积操作可以被函数 g 表示，该函数 g 的输入是词嵌入向量 $x_m, x_n$ 和它们之间的相对位置 $m-n$:

$$<f_q(x_m ,m), f_k(x_n, n)>=g(x_m, x_n, m - n)$$

旋转位置编码就是找到一个使上式成立的位置编码方式. 

出于认识的目的，我们省略复杂的数学推导，直接看 RoPE 的的结论：

存在这样一个正交矩阵：

$$\boldsymbol{R}_{\Theta,m}^d=\underbrace{\begin{pmatrix}\cos m\theta_0&-\sin m\theta_0&0&0&\cdots&0&0\\\sin m\theta_0&\cos m\theta_0&0&0&\cdots&0&0\\0&0&\cos m\theta_1&-\sin m\theta_1&\cdots&0&0\\0&0&\sin m\theta_1&\cos m\theta_1&\cdots&0&0\\\vdots&\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\0&0&0&0&\cdots&\cos m\theta_{d/2-1}&-\sin m\theta_{d/2-1}\\
0&0&0&0&\cdots&\sin m\theta_{d/2-1}&\cos m\theta_{d/2-1}\end{pmatrix}}_{\boldsymbol{W}_m}$$

其中，$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

我们可以将 query 和 key 的内积操作转换为与原始向量 $x$ 相关的以下等价形式：

$$
\boldsymbol{q}_m^\mathbf{T}\boldsymbol{k}_n=\left(\boldsymbol{R}_{\Theta,m}^d\boldsymbol{W}_q\boldsymbol{x}_m\right)^\mathbf{T}\left(\boldsymbol{R}_{\Theta,n}^d\boldsymbol{W}_k\boldsymbol{x}_n\right)=\boldsymbol{x}_m^\mathbf{T}\boldsymbol{W}_q\boldsymbol{R}_{\Theta,n-m}^d\boldsymbol{W}_k\boldsymbol{x}_n
$$

其中， $\boldsymbol{R}_{\Theta,n-m}^d=\left(\boldsymbol{R}_{\Theta,m}^d\right)^\mathbf{T}\boldsymbol{R}_{\Theta,n}^d$.

由于 $\boldsymbol{R}_{\Theta,m}^d$ 的稀疏性，直接使用矩阵乘法会浪费算力，因此代码中采用下述方式实现：

$$\boldsymbol{R}_{\Theta,m}^{d}\boldsymbol{x}=\begin{pmatrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\\vdots\\x_{d-2}\\x_{d-1}\end{pmatrix}\otimes\begin{pmatrix}\cos m\theta_{0}\\\cos m\theta_{0}\\\cos m\theta_{1}\\\cos m\theta_{1}\\\vdots\\\cos m\theta_{d/2-1}\\\cos m\theta_{d/2-1}\end{pmatrix}+\begin{pmatrix}-x_{1}\\x_{0}\\-x_{3}\\x_{2}\\\vdots\\-x_{d-1}\\x_{d-2}\end{pmatrix}\otimes\begin{pmatrix}\sin m\theta_{0}\\\sin m\theta_{0}\\\sin m\theta_{1}\\\sin m\theta_{1}\\\vdots\\\sin m\theta_{d/2-1}\\\sin m\theta_{d/2-1}\end{pmatrix}
$$

In [ ]:
# 生成旋转矩阵，dim与head_dim相同（公式里为d_k，计算方法为d_model/num_heads），end与seq_len相同
def precompute_pos_cis(dim: int,  end: int=int(32 * 1024),  theta: float=1e6):
    # 计算词向量元素两两分组后 每组元素对应的旋转角度 \theta_i  i = 0, 1, ..., d/2-1
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    # token 序列索引 t = [0, 1, ..., seq_len-1]
    t = torch.arange(end,  device=freqs.device)
    # 计算 m * \theta  维度变为 (seq_len, dim/2)
    freqs = torch.outer(t,  freqs).float()
    # pos_cis的计算结果是个复数向量  torch.polar(r,  theta) 其中 r 是模长 theta 是辐角
    # 假设 freqs = [x, y]
    # 则 freqs_cis = [cos(x) + sin(x)i, cos(y) + sin(y)i]
    pos_cis = torch.polar(torch.ones_like(freqs),  freqs)
    return pos_cis

# 应用旋转位置编码
def apply_rotary_emb(xq,  xk,  pos_cis):
    def unite_shape(pos_cis,  x):
        ndim = x.ndim
        assert 0 <= 1 < ndim
        assert pos_cis.shape == (x.shape[1],  x.shape[-1])
        # 构造新形状：仅保留 x 的第1维和最后1维，其余维度设为1， shape 的维度为 [1, seq_len, 1, head_dim//2]
        shape = [d if i == 1 or i == ndim - 1 else 1 for i,  d in enumerate(x.shape)]
        return pos_cis.view(*shape) # 调整 pos_cis 形状与 x 匹配以便广播运算

    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1],  -1,  2)) # 按组合并最后一维的实部和虚部为复数
    # print('original shape xq: {}'.format(xq.shape)) # 取消注释以查看 xq 的 shape
    # print('ajusted shape xq: {}'.format(xq_.shape)) # 取消注释查看经过复数表示后 xq 的 shape
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1],  -1,  2))
    pos_cis = unite_shape(pos_cis,  xq_)
    # print('ajusted shape pos_cis： {}'.format(pos_cis.shape)) # 取消注释以查看位置编码旋转角的形状
    # 保留实部
    xq_out = torch.view_as_real(xq_ * pos_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * pos_cis).flatten(3)
    return xq_out.type_as(xq),  xk_out.type_as(xk)

我们知道，RoPE 是在 Attention 阶段生成 Query 和 Key 向量后，对这两个向量进行位置编码的.

对于 MiniMindLM，嵌入维度为 512，注意力头数量为 8，故每一个注意力头的维度应该为 512 / 8 = 64.

我们使用固定形状的张量代表 Query 和 Key 向量，对 RoPE 的应用过程进行观察.

In [4]:
# 测试旋转位置编码的预计算和应用，简化num_heads和batch_size维度
xq,  xk = torch.randn((2,  16,  4,  64)), torch.randn((2,  16,  4,  64)) # (batch_size,  sequence_len,  num_heads,  head_dim)
pos_cis = precompute_pos_cis(64,  16) # 计算旋转位置编码的旋转角（复数表示）
print(f'pos_cis 的形状为 {pos_cis.shape}, 其中 [0, 0] 下标元素为 {pos_cis[0,  0]}')

pos_cis 的形状为 torch.Size([16, 32]), 其中 [0, 0] 下标元素为 (1+0j)


In [5]:
xq_rope,  xk_rope = apply_rotary_emb(xq,  xk,  pos_cis)
# original shape xq: torch.Size([2, 16, 4, 64])
# ajusted shape xq: torch.Size([2, 16, 4, 32])
# ajusted shape pos_cis： torch.Size([1, 16, 1, 32])
print(f'经过 RoPE 编码后的 Query 与 Key 的形状为 {xq_rope.shape},  {xk_rope.shape}')

经过 RoPE 编码后的 Query 与 Key 的形状为 torch.Size([2, 16, 4, 64]),  torch.Size([2, 16, 4, 64])


### Attention

注意力机制（Attention Mechanism）是Transformer架构的核心组件，能够有效捕捉长序列内各元素间的依赖关系. 该机制通过计算输入序列中不同位置元素间的注意力得分，对其重要性进行精准建模，使模型在处理信息时能够聚焦于关键部分，从而显著提升对长序列数据的理解与处理能力.

在 MiniMindLM 模型中, Attention Block 包含以下机制和模块:

1. GQA
2. KV Cache
5. SwiGLU

- **GQA**

分组查询注意力 (Group Querey Attention, GQA) 是对多头自注意力机制的扩展, 通过提供计算效率和模型表达能力的灵活权衡, 实现了查询头的分组.

具体来说, GQA 中将 h 个查询头分为 G 组, 每组 包含 h / G 个查询头，并共享一个公共的键和值.

![img](./images/gqa.png)

GQA 相比传统的 MHA， 减少了键和值的数量，降低了计算量和内存开销，提高了推理速度.

---

#### 一、多头注意力（MHA：Multi-Head Attention）
##### 核心定义
Transformer原生注意力机制（《Attention Is All You Need》提出），Q、K、V各有相同的H个独立头，每个头独立计算注意力，最后拼接结果，是**表达能力最强**的版本，也是MQA/GQA的优化基础。
##### 执行步骤
假设输入为$X \in \mathbb{R}^{N \times 512}$（$N$为序列长度，如句子的token数）
1. **独立投影**：用3个**独立的权重矩阵**$W_Q \in \mathbb{R}^{512 \times 512}$、$W_K \in \mathbb{R}^{512 \times 512}$、$W_V \in \mathbb{R}^{512 \times 512}$，对X做线性投影，得到**同头数**的Q、K、V：
   $Q = XW_Q \in \mathbb{R}^{N \times 512}$，$K = XW_K \in \mathbb{R}^{N \times 512}$，$V = XW_V \in \mathbb{R}^{N \times 512}$；
2. **分拆成头**：将Q、K、V各拆为$H=8$个独立的头，每个头维度为$d_k=64$，得到：
   $Q_h \in \mathbb{R}^{8 \times N \times 64}$，$K_h \in \mathbb{R}^{8 \times N \times 64}$，$V_h \in \mathbb{R}^{8 \times N \times 64}$（维度：头数×序列长度×头维度）；
3. **独立计算注意力**：8个头**互不干扰**，每个头单独执行注意力公式，得到8个注意力输出：
   $head_i = Attention(Q_i, K_i, V_i) \in \mathbb{R}^{N \times 64}$（$i=1,2,...,8$）；
4. **拼接头输出**：将8个独立的头结果按最后一维拼接，恢复为$d_{model}$维度：
   $Concat = [head_1, head_2, ..., head_8] \in \mathbb{R}^{N \times 512}$；
5. **最终投影**：用输出权重矩阵$W_O \in \mathbb{R}^{512 \times 512}$做线性变换，得到MHA最终结果：
   $Output = Concat \cdot W_O \in \mathbb{R}^{N \times 512}$。

##### 关键开销（示例参数）
- 投影参数：$W_Q+W_K+W_V = 3×512×512 = 786432$ 个参数；
- KV Cache内存：每层需缓存$8$个K和$8$个V，单层缓存规模为$2×8×N×64 = 1024N$（序列长度N越大，开销越高）；
- 注意力计算量：$8×(2×N×64×N + N×64×N) = 8×3N^2×64 = 1536N^2$（与头数成正比）。

##### 核心特点
✅ **表达能力最强**：8个头独立捕捉不同的语义/位置特征（如句法关系、语义关联），适配复杂任务；  
❌ **推理效率最低**：KV Cache内存占用高，长序列（如N=4096/8192）下内存压力大，硬件带宽消耗高；  
❌ **参数冗余**：部分任务中，多个头的特征存在重叠，完全独立的头设计造成计算/内存浪费。 

#### 二、多查询注意力（MQA：Multi-Query Attention）
##### 核心定义
为极致优化推理效率提出的变体（最早见于《Fast Transformer Decoding: One Write-Head is All You Need》），保留H个独立的Q头，但仅用1个共享的K头和1个共享的V头，所有Q头共享同一组K、V计算注意力，是**效率最高、表达能力最弱**的版本。
##### 执行步骤（基于MHA同参数：$d_{model}=512$、$H=8$、$d_k=64$）
输入仍为$X \in \mathbb{R}^{N \times 512}$，核心变化是**K/V的投影矩阵和头数**：
1. **非对称投影**：Q用**多头投影矩阵**，K/V用**单头投影矩阵**，维度匹配是关键：
   - Q投影：$W_Q \in \mathbb{R}^{512 \times 512}$（与MHA一致），$Q = XW_Q \in \mathbb{R}^{N \times 512}$；
   - K投影：$W_K \in \mathbb{R}^{512 \times 64}$（仅单头维度$d_k$），$K = XW_K \in \mathbb{R}^{N \times 64}$；
   - V投影：$W_V \in \mathbb{R}^{512 \times 64}$（仅单头维度$d_k$），$V = XW_V \in \mathbb{R}^{N \times 64}$；
2. **分拆Q头，K/V保持单头**：
   - Q拆为8个独立头：$Q_h \in \mathbb{R}^{8 \times N \times 64}$（与MHA一致）；
   - K/V不拆分：$K \in \mathbb{R}^{N \times 64}$，$V \in \mathbb{R}^{N \times 64}$（无头部维度）；
3. **共享KV计算注意力**：8个Q头**全部使用同一组K、V**，各自独立计算注意力，得到8个输出：
   $head_i = Attention(Q_i, K, V) \in \mathbb{R}^{N \times 64}$（$i=1,2,...,8$）；
4. **拼接+最终投影**：与MHA完全一致，拼接8个头→$N×512$，再通过$W_O$得到最终输出。

##### 关键开销（示例参数）
- 投影参数：$W_Q + W_K + W_V = 512×512 + 2×512×64 = 512×(512+128) = 327680$ 个参数（仅为MHA的~41.6%）；
- KV Cache内存：每层仅缓存**1个K和1个V**，单层缓存规模为$2×1×N×64 = 128N$（仅为MHA的1/8，长序列下优势极致）；
- 注意力计算量：$8×(2×N×64×N + N×64×N) = 1536N^2$（与MHA一致，计算量未减少，优化的是**内存/带宽**）。

##### 核心特点
✅ **推理效率极致**：KV Cache内存占用降至MHA的$1/H$，大幅降低硬件带宽消耗，长序列推理速度提升显著；  
✅ **参数更少**：K/V投影矩阵维度大幅减小，模型总参数量降低；  
❌ **表达能力损失大**：所有Q头共享同一组K/V，无法捕捉多样化的特征关联，复杂任务（如大模型生成、精细语义理解）中**性能明显下降**；  
❌ **泛化性弱**：单组KV难以适配不同Q头的特征需求，易出现特征混叠。

#### 三、分组查询注意力（GQA：Grouped-Query Attention）
##### 核心定义
2023年《GQA: Training Generalized Multi-Query Transformer Models from Multi-Head Checkpoints》提出的中间折中方案，融合MHA的表达能力和MQA的推理效率，将H个Q头划分为G个组，**每组内的Q头共享1组独立的K/V头**（G为组数，可灵活调整）。
##### 核心参数关系（统一约束）
- 总Q头数：$H$（固定，由$d_{model}/d_k$决定）；
- 组数：$G$（超参数，人工设定，如2/4/8）；
- 每组Q头数：$h = H/G$（必须为整数，如$H=8$、$G=4$时，$h=2$）；
- K/V头数：**等于组数G**（每组1组独立KV，无共享）。

**示例参数**（延续前文，且选工程中常用的$G=4$）：$d_{model}=512$、$H=8$、$G=4$、$h=2$、$d_k=64$。

##### 执行步骤（G=4为示例）
输入仍为$X \in \mathbb{R}^{N \times 512}$，核心是**分组划分+组内KV共享**：
1. **分组投影**：Q用多头投影矩阵，K/V用**组数G对应的投影矩阵**（头数=G）：
   - Q投影：$W_Q \in \mathbb{R}^{512 \times 512}$（与MHA一致），$Q = XW_Q \in \mathbb{R}^{N \times 512}$；
   - K投影：$W_K \in \mathbb{R}^{512 \times (G×d_k)} = \mathbb{R}^{512 \times 256}$，$K = XW_K \in \mathbb{R}^{N \times 256}$；
   - V投影：$W_V \in \mathbb{R}^{512 \times (G×d_k)} = \mathbb{R}^{512 \times 256}$，$V = XW_V \in \mathbb{R}^{N \times 256}$；
2. **分拆Q头，分拆KV为组**：
   - Q拆为8个独立头：$Q_h \in \mathbb{R}^{8 \times N \times 64}$，并按$G=4$分组：$[Q_1Q_2, Q_3Q_4, Q_5Q_6, Q_7Q_8]$（每组2个Q头）；
   - K拆为G=4个组头：$K_g \in \mathbb{R}^{4 \times N \times 64}$（每组1个K头，对应1组Q头）；
   - V拆为G=4个组头：$V_g \in \mathbb{R}^{4 \times N \times 64}$（与K头一一对应）；
3. **组内共享KV，组间独立计算**：**不同组之间KV完全独立，同组内所有Q头共享本组KV**，4个组并行计算注意力：
   - 组1：$head_1=Attention(Q_1,K_1,V_1)$、$head_2=Attention(Q_2,K_1,V_1)$；
   - 组2：$head_3=Attention(Q_3,K_2,V_2)$、$head_4=Attention(Q_4,K_2,V_2)$；
   - 组3/4：按上述规则执行，最终得到8个注意力头输出；
4. **拼接+最终投影**：与MHA、MQA完全一致，拼接8个头→$N×512$，通过$W_O$得到最终输出。

##### 关键开销（示例参数：G=4）
- 投影参数：$W_Q + W_K + W_V = 512×512 + 2×512×256 = 512×(512+512) = 524288$ 个参数（介于MHA和MQA之间）；
- KV Cache内存：每层缓存**G=4个K和4个V**，单层缓存规模为$2×4×N×64 = 512N$（为MHA的1/2、MQA的4倍，可通过调整G灵活控制）；
- 注意力计算量：$8×3N^2×64 = 1536N^2$（与MHA、MQA一致，计算量无损失，仅优化内存/带宽）。

##### 灵活调优性（G的核心作用）
GQA的精髓在于**组数G是可调节的超参数**，可实现「效率-性能」的连续权衡：
- 当$G=H$（如G=8）：每组1个Q头，K/V头数=Q头数，**GQA等价于MHA**（表达能力拉满，效率最低）；
- 当$G=1$：所有Q头归为1组，共享1组KV，**GQA等价于MQA**（效率拉满，表达能力最弱）；
- 当$1<G<H$（如G=2/4/6）：介于两者之间，是工程中最常用的选择（如GPT-4采用GQA，G取8/16）。

##### 核心特点
✅ **效率与性能平衡**：在损失少量表达能力的前提下，将KV Cache开销降至MHA的$G/H$，长序列推理效率大幅提升，且性能远优于MQA；  
✅ **灵活可控**：通过调整G适配不同硬件（低内存硬件选小G，高内存硬件选大G）和任务（简单任务选小G，复杂任务选大G）；  
✅ **无缝迁移**：可直接从MHA预训练模型的权重微调得到GQA模型，无需从零训练，工程成本低；  
✅ **计算量无损失**：仅优化KV的存储和带宽，注意力计算量与MHA一致，不影响训练/推理的计算速度；  
❌ **存在轻微表达损失**：相比MHA，组内Q头共享KV会损失少量特征多样性，但工程中可通过合理选G将损失降至可接受范围（如G=4时，性能与MHA几乎持平）。

#### 四、三者核心维度对比表（统一参数：H=8、d_k=64、G=4）
为了更直观的对比，将三者的核心差异汇总为表格，**红色为核心差异点**：

|对比维度|多头注意力（MHA）|多查询注意力（MQA）|分组查询注意力（GQA）|
| ---- | ---- | ---- | ---- |
|Q头数|8（独立）|8（独立）|8（独立，分4组）|
|**K/V头数**|**8（与Q一一对应）**|**1（所有Q共享）**|**4（与组数G一致，组内共享）**|
|投影参数规模|786432（最大）|327680（最小）|524288（中间）|
|**KV Cache内存（单层）**|**1024N（最大）**|**128N（最小）**|**512N（中间）**|
|注意力计算量|$1536N^2$|$1536N^2$|$1536N^2$（三者一致）|
|表达能力|⭐⭐⭐⭐⭐（最强）|⭐⭐（最弱）|⭐⭐⭐⭐（接近MHA）|
|推理效率（内存/带宽）|⭐（最低）|⭐⭐⭐⭐⭐（最高）|⭐⭐⭐⭐（接近MQA）|
|参数冗余度|高（头间特征重叠）|无（单KV）|中（组内无重叠，组间少量重叠）|
|工程适配性|适合训练/小序列推理|适合极致效率的简单任务|**大模型长序列推理（最优选择）**|
|典型应用|Transformer基础版、小模型|轻量级端侧模型、快速推理场景|GPT-4、PaLM、LLaMA2（大模型主流）|

#### 五、核心总结
1. **设计初衷**：MHA是基础，追求表达能力；MQA为极致效率牺牲表达；GQA是工程化的最优折中，解决大模型长序列推理的「内存瓶颈」；
2. **核心优化点**：三者的**注意力计算量完全一致**，MQA/GQA的优化核心是**减少KV Cache的内存占用和硬件带宽消耗**（推理阶段的核心开销），而非计算速度；
3. **GQA的核心价值**：成为大模型的主流选择，本质是解决了「大模型长序列推理时，MHA内存不够用，MQA性能损失太大」的工程痛点；
4. **选型原则**：
   - 训练阶段/小序列推理/复杂任务：选MHA，保证表达能力；
   - 端侧模型/简单任务/极致效率需求：选MQA；
   - 大模型/长序列推理（如N=4096/8192）/兼顾性能与效率：选GQA（重点调优组数G）。


---

- **KV Cache**

在语言模型生成文本的过程中，每生成一个新的 token，模型都需要计算注意力得分，以确定当前位置与之前所有位置的相关性.

比如以下内容：

1. *seq = \[tok1]:*

   attn_11 = softmax(Q1 * K1.T / sqrt(dim)) * V1
   
3. *seq = \[tok1, tok2]:*

   attn_11 = softmax(Q1 * K1.T / sqrt(dim)) * V1, attn_12 = 0 (masked)
   
   attn_21 = softmax(Q2 * K1.T / sqrt(dim)) * V1, attn_22 = softmax(Q2 * K2.T / sqrt(dim)) * V2
4. *seq = \[tok1, tok2, tok3]:*

   attn_11 = softmax(Q1 * K1.T / sqrt(dim)) * V1, attn_12 = 0 (masked), attn_13 = 0 (masked)
   
   attn_21 = softmax(Q2 * K1.T / sqrt(dim)) * V1, attn_22 = softmax(Q2 * K2.T / sqrt(dim)) * V2, attn_23 = 0 (masked)
   
   attn_31 = softmax(Q3 * K1.T / sqrt(dim)) * V1, attn_32 = softmax(Q3 * K2.T / sqrt(dim)) * V2, attn_33 = softmax(Q3 * K3.T / sqrt(dim)) * V3
5.  ··· ···

不难发现，大模型生成一个 token 后的注意力计算中，总会用到 token 序列的历史 KV 值，导致重复计算，KV Cache 的设计正是为了通过缓存历史 KV 值，节省计算开销.

KV Cache 能够有效压缩大模型推理时的显存占用.

- **SwiGLU**

SwiGLU 是一种在深度学习中用于神经网络架构的激活函数变体：

$$\text{SwiGLU}(x, W, V, b, c)=\text{Swish}_1(xW+b)\otimes(xV+c)$$

与传统的 ReLU 激活函数相比，SwiGLU 具有更好的平滑性和非线性表达能力，由于其门控机制，在处理信息筛选和流动方面有独特的优势.

---

| 特性                | ReLU                          | SwiGLU                        |
|---------------------|-------------------------------|-------------------------------|
| **平滑性**          | 非平滑，$x=0$处不可微         | 处处平滑可微，无断点          |
| **梯度传播**        | $x<0$时梯度为0，易梯度消失    | 全区间有非零梯度，梯度传播更稳定 |
| **信息处理**        | 硬截断，丢弃所有负向信息      | 门控机制，自适应筛选/增强正负信息 |
| **非线性表达**      | 简单分段线性，表达能力有限    | 平滑非线性+门控，表达能力极强  |
| **计算复杂度**      | 极低，仅比较和取最大值        | 略高，需两次线性变换+Swish+按元素乘 |
| **适用场景**        | 浅层/中型网络，追求计算效率    | 深层网络（如Transformer），追求性能上限 |

---

In [6]:
from typing import Any,  Optional,  Tuple,  List
import torch.nn as nn
import math

# 使得 KV 头数适应 Query 头数
def repeat_kv(x: torch.Tensor,  n_rep: int) -> torch.Tensor:
    bs,  slen,  n_kv_heads,  head_dim = x.shape
    if n_rep == 1: # 如果KV头数已经和Query头数相同，则不进行重复
        return x
    return (
        x[:,  :,  :,  None,  :]
        .expand(bs,  slen,  n_kv_heads,  n_rep,  head_dim)  # 在新维度上扩展
        .reshape(bs,  slen,  n_kv_heads * n_rep,  head_dim)  # 合并头数维度
    )

class Attention(nn.Module):
    def __init__(self,  args: LMConfig):
        super().__init__()
        # 1. 配置头数和维度
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads # 默认KV头数与Query头数相同
        assert args.n_heads % self.n_kv_heads == 0
        self.n_local_heads = args.n_heads  # Query头数
        self.n_local_kv_heads = args.n_kv_heads  # KV头数
        self.n_rep = self.n_local_heads // self.n_local_kv_heads  # KV头数重复次数
        self.head_dim = args.dim // args.n_heads  # 每个头的维度，d_k = d_model / num_heads
        # 2. 线性投影层（QKV映射+输出映射）
        self.wq = nn.Linear(args.dim,  args.n_heads * self.head_dim,  bias=False)  # Query 映射
        self.wk = nn.Linear(args.dim,  args.n_kv_heads * self.head_dim,  bias=False)  # Key 映射
        self.wv = nn.Linear(args.dim,  args.n_kv_heads * self.head_dim,  bias=False)  # Value 映射
        self.wo = nn.Linear(args.n_heads * self.head_dim,  args.dim,  bias=False)  # 输出映射
        # 3. Dropout 层
        self.attn_dropout = nn.Dropout(args.dropout)  # 注意力权重 Dropout
        self.resid_dropout = nn.Dropout(args.dropout)  # 输出 Dropout
        self.dropout = args.dropout  # 用于 Flash Attention 的 dropout 概率
        # 4. Flash Attention支持（加速计算）
        self.flash = hasattr(torch.nn.functional,  'scaled_dot_product_attention') and args.flash_attn
        # 5. 注意力掩码（因果掩码）
        mask = torch.full((1,  1,  args.max_seq_len,  args.max_seq_len),  float("-inf"))  # 初始化为负无穷
        mask = torch.triu(mask,  diagonal=1)  # 上三角矩阵（不含主对角线）设为负无穷
        self.register_buffer("mask",  mask,  persistent=False)  # 注册为缓冲区，不作为模型参数保存

    def forward(self, 
               x: torch.Tensor, 
               pos_cis: torch.Tensor, 
               past_key_value: Optional[Tuple[torch.Tensor,  torch.Tensor]] = None, 
               use_cache=False):
        bsz,  seq_len,  _ = x.shape  # x 的形状为 (batch_size,  seq_len,  dim)
        # 1. 线性映射得到 Q, K, V
        xq,  xk,  xv = self.wq(x),  self.wk(x),  self.wv(x)  # 线性映射
        # 维度变换：(bsz, seq_len, dim) → (bsz, seq_len, num_heads, head_dim)
        xq = xq.view(bsz,  seq_len,  self.n_local_heads,  self.head_dim)
        xk = xk.view(bsz,  seq_len,  self.n_local_kv_heads,  self.head_dim)
        xv = xv.view(bsz,  seq_len,  self.n_local_kv_heads,  self.head_dim)
        xq,  xk = apply_rotary_emb(xq,  xk,  pos_cis)  # Q 和 K 应用 RoPE 编码
        # 2. 处理缓存的 past_key_value
        if past_key_value is not None:
            # 拼接历史KV和当前KV，维度在seq_len方向扩展
            xk = torch.cat([past_key_value[0],  xk],  dim=1)
            xv = torch.cat([past_key_value[1],  xv],  dim=1)
        past_kv = (xk,  xv) if use_cache else None  # 更新缓存
        xq,  xk,  xv = (
            # 维度变换：(bsz, seq_len, num_heads, head_dim) → (bsz, num_heads, seq_len, head_dim)
            xq.transpose(1,  2), 
            repeat_kv(xk,  self.n_rep).transpose(1,  2),   # 使得 KV 头数适应 Query 头数
            repeat_kv(xv,  self.n_rep).transpose(1,  2)
        )
        # 3. 计算注意力权重（Scaled Dot Production）
        if self.flash and seq_len != 1:
            dropout_p = self.dropout if self.training else 0.0
            output = F.scaled_dot_product_attention(
                xq,  xk,  xv, 
                attn_mask=None,   # 因果掩码已内置于 Flash Attention 中
                dropout_p=dropout_p, 
                is_causal=True   # 启用因果掩码
            )
        else:
            scores = (xq @ xk.transpose(-2,  -1)) / math.sqrt(self.head_dim)  # Q×K^T / √d_k（缩放点积）
            scores += self.mask[:,  :,  :seq_len,  :seq_len]   # 应用因果掩码
            scores = F.softmax(scores.float(),  dim=-1).type_as(xq)   # 注意力权重（softmax）
            scores = self.attn_dropout(scores)   # 注意力权重 Dropout
            output = scores @ xv  # 注意力加权值（加权求和）
        # 4. 输出映射和 Dropout
        output = output.transpose(1,  2).reshape(bsz,  seq_len,  -1)   # 维度变换回 (bsz, seq_len, dim)
        output = self.resid_dropout(self.wo(output))   # 输出映射和 Dropout
        return output,  past_kv

同样的，我们假设一批 batch size = 4， seq len = 16 的 token 序列通过这个 Attention 块，在输入前，它的 input id 会被投影到 dim = 512 维.

In [7]:
# 创建 Config，为了简单起见，我们设定 num_layers = 2
LMConfig_Dense = LMConfig(n_layers=2)

In [8]:
attn = Attention(LMConfig_Dense)
x = torch.randn((4,  16,  512)) # (batch_size, seq_len, d_model)
pos_cis = precompute_pos_cis(64,  16) # (head_dim, batch_size) 其中 head_dim = d_model / num_heads
output,  past_kv = attn(x,  pos_cis=pos_cis,  use_cache=True)
print(f'输入张量 x ：size = {x.shape}，RoPE 旋转角： size = {pos_cis.shape}')
print(f'输出 output: size = {output.shape},  kv_cache 基本信息：size_key = {past_kv[0].shape}, size_value = {past_kv[1].shape}')

输入张量 x ：size = torch.Size([4, 16, 512])，RoPE 旋转角： size = torch.Size([16, 32])
输出 output: size = torch.Size([4, 16, 512]),  kv_cache 基本信息：size_key = torch.Size([4, 16, 2, 64]), size_value = torch.Size([4, 16, 2, 64])


In [9]:
del attn  # 删除 attn 以释放内存

### FeedForward Network

前馈神经网络接收来自注意力机制层的输出结果，随后对该输出执行进一步的线性变换. 通过这种方式，网络能够深入挖掘并捕获更为复杂、抽象的特征.

In [10]:
class FeedForward(nn.Module):
    def __init__(self,  config: LMConfig):
        super().__init__()
        # 1. 计算隐藏层维度 hidden_dim
        if config.hidden_dim is None:
            hidden_dim = 4 * config.dim
            hidden_dim = int(2 * hidden_dim / 3)
            # 向上取整到最近的 multiple_of 的倍数（通常为 256 或 128，利于硬件计算）
            config.hidden_dim = config.multiple_of * ((hidden_dim + config.multiple_of - 1) // config.multiple_of)
        # 2. 定义三个线性层
        self.w1 = nn.Linear(config.dim,  config.hidden_dim,  bias=False)  # 分支1：生成激活值
        self.w2 = nn.Linear(config.hidden_dim,  config.dim,  bias=False)  # 输出层：映射回原维度
        self.w3 = nn.Linear(config.dim,  config.hidden_dim,  bias=False)  # 分支2：生成门控值
        # 3. Dropout 层
        self.dropout = nn.Dropout(config.dropout)

    def forward(self,  x):
        return self.dropout(self.w2(F.silu(self.w1(x)) * self.w3(x)))  # 对应用 SiLU 激活函数后的门控机制

设置输入 x 为 batch size = 4，seq len = 16 的 token 序列投影向量，观察 x 在 MiniMind Block 的前向传播过程.

In [11]:
ffn = FeedForward(LMConfig_Dense)  # 创建 FeedForward 实例
x = torch.randn((4,  16,  512))
output = ffn(x)
print(f'给定输入 x: size = {x.shape} 下的输出 output：size = {output.shape}')

给定输入 x: size = torch.Size([4, 16, 512]) 下的输出 output：size = torch.Size([4, 16, 512])


In [12]:
del ffn  # 删除 ffn 以释放内存

这就是 Transformer 结构的精妙之处，张量在模型内部进行了各种复杂的投影变形，但是输入输出的张量形状不发生改变!

### MiniMind Block

到目前为止,  已经完成了 Attention Layer 和 FeedForward Layer 的构建, 所有必须的组件都已经具备, 我们着手构建一个 MiniMind Block

In [13]:
class MiniMindBlock(nn.Module):
    def __init__(self,  layer_id: int,  config: LMConfig):
        super().__init__()
        self.n_heads = config.n_heads
        self.dim = config.dim
        self.head_dim = config.dim // config.n_heads
        self.attention = Attention(config)  # 注意力机制

        self.layer_id = layer_id  # layer id 指向其维护的 KV Cache
        self.attention_norm = RMSNorm(config.dim,  eps=config.norm_eps)
        self.ffn_norm = RMSNorm(config.dim,  eps=config.norm_eps)
        self.feed_forward = FeedForward(config)

    def forward(self,  x,  pos_cis,  past_key_value=None,  use_cache=False):
        # 1. 注意力计算：先归一化，再进入注意力层
        h_attn,  past_kv = self.attention(
            self.attention_norm(x),  # pre-norm：先归一化再输入注意力层
            pos_cis, 
            past_key_value=past_key_value, 
            use_cache=use_cache
        )
        # 2. 残差连接：注意力输出 + 原始输入（注意力前的输入）
        h = x + h_attn 
        # 3. 前馈网络：先归一化，再进入前馈层，然后残差连接
        out = h + self.feed_forward(self.ffn_norm(h))  
        return out,  past_kv

我们依然设置输入 x 为 batch size = 4，seq len = 16 的 token 序列投影向量，观察 x 在 MiniMind Block 的前向传播过程.

In [14]:
# 测试 MiniMindBlock，为了简单起见，我们只创建一个 Block
miniblock = MiniMindBlock(1,  LMConfig_Dense)
x = torch.randn((4,  16,  512))
pos_cis = precompute_pos_cis(64,  16)
out,  past_kv = miniblock(x,  pos_cis,  use_cache=True)
print(f'输出 output 信息: size = {out.shape}\n该 Block 维护的 KV Cache 信息：size_key =  {past_kv[0].shape}, size_value = {past_kv[1].shape}')

输出 output 信息: size = torch.Size([4, 16, 512])
该 Block 维护的 KV Cache 信息：size_key =  torch.Size([4, 16, 2, 64]), size_value = torch.Size([4, 16, 2, 64])


In [15]:
del miniblock

### MiniMindLM (Dense)

以 MiniMind Block 为基本组件, 我们对 MiniMindLM 进行最后组装！

In [16]:
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

class MiniMindLM(PreTrainedModel):
    config_class = LMConfig

    def __init__(self,  params: LMConfig = None):
        self.params = params or LMConfig()
        super().__init__(self.params)
        self.vocab_size,  self.n_layers = params.vocab_size,  params.n_layers
        # 映射：词表维度 -> 嵌入维度
        self.tok_embeddings = nn.Embedding(params.vocab_size,  params.dim)
        self.dropout = nn.Dropout(params.dropout)
        self.layers = nn.ModuleList([MiniMindBlock(1,  params) for _ in range(self.n_layers)])
        self.norm = RMSNorm(params.dim,  eps = params.norm_eps)
        self.output = nn.Linear(params.dim,  params.vocab_size,  bias=False)  # 将嵌入向量映射回词表，得到每个 token 的预测概率
        self.tok_embeddings.weight = self.output.weight  # 词嵌入与输出层权重共享
        # 预计算 RoPE 旋转角
        self.register_buffer(  
            "pos_cis", 
            precompute_pos_cis(dim = params.dim // params.n_heads,  theta=params.rope_theta), 
            persistent=False  # 不将该缓冲区保存为模型参数
        )
        self.OUT = CausalLMOutputWithPast()   # 输出容器：用于封装模型输出（logits、KV Cache 等）

    def forward(self, 
               input_ids: Optional[torch.Tensor] = None, 
               past_key_values: Optional[List[Tuple[torch.Tensor,  torch.Tensor]]] = None, 
               use_cache: bool = False, 
               **args):
        past_key_values = past_key_values or [None] * len(self.layers) # 初始化 KV Cache：若未传入，初始化为与层数等长的 None 列表
        start_pos = args.get('start_pos',  0)  # 获取当前输入序列的起始位置
        h = self.dropout(self.tok_embeddings(input_ids))  # 词嵌入与 Dropout
        pos_cis = self.pos_cis[start_pos: start_pos + input_ids.size(1)]  # 获取对应位置的 RoPE 旋转角
        past_kvs = [] # KV Cache 列表 按照索引维护对应 MiniMind Block 模块的 KV Cache
        print(f'====> forward propagation started, num_minimind_blocks = {len(self.layers)}')
        for l,  layer in enumerate(self.layers):
            print(f'------------> entering minimind block: id = {l}')
            h,  past_kv = layer(
                h,  pos_cis, 
                past_key_value=past_key_values[l], 
                use_cache=use_cache
            )
            print(f'<------------ finished, size_cache_k = {past_kv[0].shape}, size_cache_v = {past_kv[1].shape}')
            past_kvs.append(past_kv)
        print(f'<==== forward propagation completed, num_kv_cache = {len(past_kvs)}\n')
        logits = self.output(self.norm(h))  # 归一化后映射回词表维度，得到每个 token 的预测 logits
        aux_loss = 0 # MoE 辅助损失，我们在 Dense Model 中不纳入考虑
        self.OUT.__setitem__('logits',  logits) # token 分类 logits
        self.OUT.__setitem__('aux_loss',  aux_loss) # MoE 辅助损失
        self.OUT.__setitem__('past_key_value',  past_kvs) # 当前序列的历史 KV_cache 列表
        return self.OUT

    @torch.inference_mode()  # 推理模式下禁用梯度计算，节省内存和计算资源
    def generate(self,  input_ids,  eos_token_id=2,  max_new_tokens=512,  temperature=0.75,  top_p=0.90, 
                stream=False,  rp=1,  use_cache=True,  pad_token_id=0,  **args):
        # 流式生成：返回生成器，逐 token 输出（适合实时展示）
        if stream:
            return self._stream(input_ids,  eos_token_id,  max_new_tokens,  temperature,  top_p,  rp,  use_cache,  **args)
        # 非流式生成：批量生成，最终返回完整序列
        generated = []
        for i in range(input_ids.size(0)):  # 遍历 batch 中的每个样本
            non_pad = input_ids[i][input_ids[i] != pad_token_id].unsqueeze(0)  # 去除 padding token
            out = self._stream(non_pad,  eos_token_id,  max_new_tokens,  temperature,  top_p,  rp,  use_cache,  **args)
            tokens_list = [tokens[:,  -1:] for tokens in out]  # 获取生成的新 token 列表
            print(f'new tokens list :{tokens_list}\n')
            gen = torch.cat(tokens_list,  dim=-1) if tokens_list else non_pad  # 拼接生成的新 token
            full_sequence = torch.cat([non_pad,  gen],  dim=-1)  # 拼接原始非 padding 序列与生成序列
            generated.append(full_sequence)  # 添加到生成列表
        max_length = max(seq.size(1) for seq in generated)  # 计算最大序列长度以便 padding
        generated = [
            torch.cat([seq,  torch.full((1,  max_length - seq.size(1)),  pad_token_id,  dtype=seq.dtype,  device=seq.device)], dim=-1) 
            for seq in generated
        ]  # 对齐序列长度，generated的每个元素shape均为 (1, max_length)
        return torch.cat(generated,  dim=0)  # 拼接为最终输出张量，维度为 (batch_size,  max_length)

    def _stream(self,  input_ids,  eos_token_id,  max_new_tokens,  temperature,  top_p,  rp,  use_cache,  **args):
        start,  first_seq,  past_kvs = input_ids.shape[1],  True,  None
        new_token_idx = 0 #  new token 计数器
        while input_ids.shape[1] < max_new_tokens - 1:  # 控制生成长度
            print(f'gernerating new token: idx = {start + new_token_idx}')
            if first_seq or not use_cache: # 若第一次生成序列 or 无 KV Cache,  每次生成传入整个 token id 序列
                out,  first_seq = self(input_ids,  past_key_values=past_kvs,  use_cache=use_cache,  **args),  False
            else: # 若非第一次生成 and 有 KV Cache, 每次传入最后一个 token id 与 KV Cache 进行推理加速
                out = self(input_ids[:,  -1:],  past_key_values=past_kvs,  use_cache=use_cache, 
                           start_pos=input_ids.shape[1] - 1,  **args)
            logits,  past_kvs = out.logits[:,  -1,  :],  out.past_key_values # logits.shape: (batch_size,  seq_len,  embed_dim), 获取最后一位 logits
            logits[:,  list(set(input_ids.tolist()[0]))] /= rp # 对生成 token 进行惩罚, 降低后续重复生成几率
            logits /= (temperature + 1e-9) # 调整温度, 控制生成多样性
            if top_p is not None and top_p < 1.0: # top-p 采样
                sorted_logits,  sorted_indices = torch.sort(logits,  descending=True,  dim=-1)
                sorted_probs = F.softmax(sorted_logits,  dim=-1)  # 计算概率分布
                cumulative_probs = torch.cumsum(sorted_probs,  dim=-1)  # 计算累积概率，用于筛选
                sorted_indices_to_remove = cumulative_probs > top_p  # 标记需要移除的 token
                sorted_indices_to_remove[:,  1:] = sorted_indices_to_remove[:,  :-1].clone() # 向右移动一位
                sorted_indices_to_remove[:,  0] = False  # 确保至少保留一个 token
                indices_to_remove = sorted_indices_to_remove.scatter(1,  sorted_indices,  sorted_indices_to_remove)  # 恢复原始索引顺序
                logits[indices_to_remove] = -float('Inf')  # 将不保留的 token logits 设为负无穷
            input_ids_next = torch.multinomial(F.softmax(logits,  dim=-1),  num_samples=1) # 从保留的 token 中采样
            input_ids = torch.cat((input_ids,  input_ids_next),  dim=1)  # 将新 token 拼接到输入序列后，准备下一轮循环
            new_token_idx += 1
            yield input_ids[:,  start:]
            if input_ids_next.item() == eos_token_id:  # 遇到结束 token 则停止生成
                break

接下来，我们设置一条长度为 4 的 token 序列，使用 MiniMindLM 对其执行一次前向传播，观察传播过程与返回值.

In [17]:
MiniMind_Dense = MiniMindLM(LMConfig_Dense)  # 创建 MiniMindLM 实例
input_ids = torch.Tensor([1,  3,  5,  7]).long().reshape(1,  4)
OUT = MiniMind_Dense(input_ids,  use_cache=True)
print(f'返回 logits：size = {OUT.logits.shape}, 返回 aux_loss: {OUT.aux_loss},  返回 KV Cache List： len = {len(OUT.past_key_value)}')

====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

返回 logits：size = torch.Size([1, 4, 6400]), 返回 aux_loss: 0,  返回 KV Cache List： len = 2


In [18]:
# 我们让 MiniMind 根据我们设计的 四个输入 token 生成输出
out = MiniMind_Dense.generate(input_ids,  max_new_tokens=8,  use_cache=True)
print(f'生成结果：{out}')

gernerating new token: idx = 4
====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

gernerating new token: idx = 5
====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 1, 2, 64]), size_cache_v = torch.Size([1, 1, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 1, 2, 64]), size_cache_v = torch.Size([1, 1, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

gernerating new token: idx = 6
====> forward propagation started, num_minimind_blocks = 2
--------

In [19]:
del MiniMind_Dense

## Minimind MoE Model

作者提供了 MiniMind MoE Model 的可视化.

![image](./images/LLM-structure-moe.png)

可以看到，Dense Model 和 MoE Model 的差异在于 FFN 层. MoE 模型将稠密连接的 FFN 层置换为 M x Expert 层，每次前向传播时只激活部分 Expert.

其组成可以分为以下部分：

- Experts: MoE 架构单元，每个专家本质上是一个独立的神经网络模块，负责处理特定类型或范围的数据.
- Router： 控制信息流动，决定每次前向传播激活的 Experts 模块以及输入数据在这些模块的分配组合.

在 MoE 网络中，为了平衡专家的重要性，我们需要关注路由，它是决定在特定时间选择哪些专家的重要组件.

### 辅助损失

为了让训练过程中实现专家的更均匀分布，辅助损失（又称负载均衡损失）被添加到网络的常规损失中，它增加了一个约束，迫使专家具有相等的重要性.

假设有输入序列 \[What is Mixture of Experts], *Prob* (·) 表示每一个 token 激活的专家概率分布:

- **Step 1**：在整个批次中对每个专家的路由值进行求和.

   $$Importance \, per \, expert = Prob (What) + Prob (is) + Prob (Mixture) + Prob (of) + Prob (Experts)$$

   这个指标反映了 batch 维度上每个专家的重要性分数.

- **Step 2**: 计算变异系数

   我们希望专家之间的重要性尽可能靠近，为了衡量专家得分之间的差异程度，引入变异系数指标（Coefficient Variation, CV）

   $$Coeifficient \, Variation (CV) = \dfrac{standard \, deviation (\sigma)}{mean(\mu)}$$

   如果专家的重要性分数相似，变异系数会降到很低（这是我们期望的）

- **Step 3**: 计算负载均衡损失

   $$uxiliary Loss = \alpha * CV$$

   其中 $\alpha$ 是缩放系数.

## MoE Gate

MoE 门控单元决定每次前向传播激活的 Experts 模块及其权重，同时计算 MoE 辅助损失.

In [20]:
class MoEGate(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()
        self.config = config
        self.top_k = config.num_experts_per_tok  # 每个 token 路由的专家数量 (K) ，即选取概率最高的 K 个专家
        self.n_routed_experts = config.n_routed_experts  # 总专家数量

        self.scoring_func = config.scoring_func # 评分函数
        self.alpha = config.aux_loss_alpha # 辅助损失的 alpha 参数
        self.seq_aux = config.seq_aux # 是否启用序列辅助损失

        self.norm_topk_prob = config.norm_topk_prob  # 对选定的 top-k 概率进行归一化
        self.gating_dim = config.dim  # Embedding 维度
        # 门控权重矩阵：[总专家数, 隐藏层维度]，用于计算token到专家的评分
        self.weight = nn.Parameter(torch.empty((self.n_routed_experts, self.gating_dim))) # 混合专家层 -> embeding dim 层
        self.reset_parameter()  # 初始化权重参数

    # kaiming_uniform 初始化
    def reset_parameter(self):
        import torch.nn.init as init
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))

    def forward(self, hidden_states):
        bsz, seq_len, h = hidden_states.shape  # hidden_states的形状为 (batch_size,  seq_len,  hidden_dim)
        hidden_states = hidden_states.view(-1, h) # (total_tokens, hidden_dim) 其中 total_tokens = bsz * seq_len
        logits = F.linear(hidden_states, self.weight, None) # logits = hidden_states · self.weight.T，形状为 (total_tokens, n_routed_experts)
        # print(f'shape of logits computed: {logits.shape}') # (total_tokens, n_routed_experts)

        # 计算专家评分
        if self.scoring_func == 'softmax':
            scores = logits.softmax(dim=-1)
        else:
            raise NotImplementedError(f'insupportable scoring function for MoE gating: {self.scoring_func}')

        # 选择 top-k 专家及其对应权重，形状为 (total_tokens, top_k)
        topk_weight, topk_idx = torch.topk(scores, k=self.top_k, dim=-1, sorted=False)  
        # 对选定的 top-k 概率进行归一化
        if self.top_k > 1 and self.norm_topk_prob: 
            denominator = topk_weight.sum(dim=-1, keepdim=True) + 1e-20
            topk_weight = topk_weight / denominator

        # 计算辅助损失（负载均衡损失）
        if self.training and self.alpha > 0.0:
            scores_for_aux = scores  # 保存原始分数以计算辅助损失，用于计算每个专家的选择概率
            aux_topk = self.top_k  # 每个 token 路由的专家数量 (K)
            topk_idx_for_aux_loss = topk_idx.view(bsz, -1) # (total_tokens, top_k) -> (batch_size, seq_len * top_k)
            if self.seq_aux:
                # 启用序列辅助损失，则计算每个序列中每个专家被选中的频率
                scores_for_seq_aux = scores_for_aux.view(bsz, seq_len, -1)  # 重塑为 (batch_size, seq_len, n_routed_experts)
                ce = torch.zeros(bsz, self.n_routed_experts, device=hidden_states.device)  # 用于统计专家选择频率的张量，形状为 (batch_size, n_routed_experts)
                # 根据索引将 torch.ones 累加到 ce 中，并执行归一化
                ce.scatter_add_(1, topk_idx_for_aux_loss,
                                torch.ones(bsz, seq_len * aux_topk, device=hidden_states.device)).div_(
                    seq_len * aux_topk / self.n_routed_experts)
                # 计算负载均衡损失
                aux_loss = (ce * scores_for_seq_aux.mean(dim=1)).sum(dim=1).mean() * self.alpha
            else:
                # 未启用序列辅助损失，则计算整体专家选择频率
                mask_ce = F.one_hot(topk_idx_for_aux_loss.view(-1), num_classes=self.n_routed_experts)
                ce = mask_ce.float().mean(0)  # 计算每个专家被选中的频率，形状为 (n_routed_experts,)
                Pi = scores_for_aux.mean(0)  # 计算每个专家的平均选择概率，形状为 (n_routed_experts,)
                fi = ce * self.n_routed_experts  # 计算负载均衡因子，大于1表示该专家被过度使用，小于1表示使用不足
                aux_loss = (Pi * fi).sum() * self.alpha  # 计算最终的辅助损失
        else:
            aux_loss = 0
        return topk_idx, topk_weight, aux_loss

接下来，我们设置一个 batch size = 4, seq len =16, emb dim = 512 的输入向量，在 MoE 门控单元前向传播进行观察

In [21]:
# 重新创建一个 LMConfig 使得 use MoE 为 True
LMConfig_MoE = LMConfig(n_layers=2, use_moe=True)

In [22]:
gate = MoEGate(LMConfig_MoE)  # 创建 MoEGate 实例
hidden_states = torch.randn((4, 16, 512))
topk_idx, topk_weight, aux_loss = gate(hidden_states) 
print(f'shape: topk_idx = {topk_idx.shape}, topk_weight = {topk_weight.shape}, aux_loss = {aux_loss}')
print(f'token 0 选择的专家：idx = {topk_idx[0]}, weight = {topk_weight[0]}')
print(f'辅助损失：aux_loss = {aux_loss}')

shape: topk_idx = torch.Size([64, 2]), topk_weight = torch.Size([64, 2]), aux_loss = 0.10560254007577896
token 0 选择的专家：idx = tensor([0, 2]), weight = tensor([0.5384, 0.4616], grad_fn=<SelectBackward0>)
辅助损失：aux_loss = 0.10560254007577896


In [23]:
del gate

### MoE Feed Forward NetWork

完成 MoE 门控单元的设计后，我们可以对 MoE 前向传播网络进行重新设计.

In [24]:
class MoEFeedForward(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()
        self.config = config
        # 可选专家层，每个专家都是一个独立的前馈网络
        self.experts = nn.ModuleList([
            FeedForward(config)
            for _ in range(config.n_routed_experts)
        ])
        self.gate = MoEGate(config)  # 门控机制
        # 共享专家层，弥补专家网络可能带来的信息损失
        if config.n_shared_experts is not None:
            self.shared_experts = FeedForward(config)

    def forward(self, x):
        identity = x  # 残差连接的原始输入
        orig_shape = x.shape  # 保存原始形状以便后续恢复
        bsz, seq_len, _ = x.shape
        # 使用门控机制选择专家
        topk_idx, topk_weight, aux_loss = self.gate(x)
        x = x.view(-1, x.shape[-1])  # 重塑为(total_tokens, hidden_dim) = (bsz * seq_len, hidden_dim)
        flat_topk_idx = topk_idx.view(-1)  # 重塑为(total_tokens * top_k, )
        # 训练模式加权求和
        if self.training:
            # 训练模式下，重复输入数据
            x = x.repeat_interleave(self.config.num_experts_per_tok, dim=0)  # (total_tokens * top_k, hidden_dim)
            y = torch.empty_like(x, dtype=torch.float16)  # 初始化输出张量，确保类型一致
            # 将输入数据中选择第 i 个专家的部分输入到该专家网络中进行处理，并将输出结果存储到 y 中对应位置的元素
            for i, expert in enumerate(self.experts):
                y[flat_topk_idx == i] = expert(x[flat_topk_idx == i]).to(y.dtype)  #  确保类型一致
            y = (y.view(*topk_weight.shape, -1) * topk_weight.unsqueeze(-1)).sum(dim=1) # 加权求和
            y = y.view(*orig_shape) # 恢复原始形状
        else:
            # 推理模式下，只选择最优专家
            y = self.moe_infer(x, flat_topk_idx, topk_weight.view(-1, 1)).view(*orig_shape)
        # 添加共享专家的输出
        if self.config.n_shared_experts is not None:
            y = y + self.shared_experts(identity)  # 残差连接
        self.aux_loss = aux_loss
        return y

    @torch.no_grad()
    def moe_infer(self, x, flat_expert_indices, flat_expert_weights):
        expert_cache = torch.zeros_like(x)
        idxs = flat_expert_indices.argsort() # 对专家索引进行排序，便于按专家分组处理，例如 [3,1,2,3,0,1] -> argsort -> [4,1,5,2,0,3]
        tokens_per_expert = flat_expert_indices.bincount().cpu().numpy().cumsum(0) #  [1, 3, 1, 2] -> bincount -> [0, 2, 1, 1] -> cumsum -> [0, 2, 3, 4]
        token_idxs = idxs // self.config.num_experts_per_tok  # 计算每个token对应的专家索引，因为每个token会被路由到top_k个专家，所以需要除以num_experts_per_tok
        # 例如当tokens_per_expert=[6, 15, 20, 26, 33, 38, 46, 52]
        # 当token_idxs=[3, 7, 19, 21, 24, 25,  4,  5,  6, 10, 11, 12...]
        # 意味着当token_idxs[:6] -> [3,  7, 19, 21, 24, 25]位置的token都由专家0处理，token_idxs[6:15]位置的token都由专家1处理......
        for i, end_idx in enumerate(tokens_per_expert):
            start_idx = 0 if i == 0 else tokens_per_expert[i - 1]
            if start_idx == end_idx:
                continue
            expert = self.experts[i]  # 获取第 i 个专家网络
            exp_token_idx = token_idxs[start_idx:end_idx]  # 获取分配给该专家的 token 索引
            expert_tokens = x[exp_token_idx]  # 提取这些 token 的输入特征
            expert_out = expert(expert_tokens).to(expert_cache.dtype)  # 专家网络前向计算，确保类型一致
            expert_out.mul_(flat_expert_weights[idxs[start_idx:end_idx]]) # 专家输出 = 专家原始输出 × 对应权重
            # 使用 scatter_add_ 进行 sum 操作
            expert_cache.scatter_add_(0, exp_token_idx.view(-1, 1).repeat(1, x.shape[-1]), expert_out)

        return expert_cache

接下来，我们设置一个 batch size = 4, seq len =16, emb dim = 512 的输入向量，在 MoE 门控单元前向传播进行观察

In [25]:
moe_ffn = MoEFeedForward(LMConfig_MoE).eval()  # 创建 MoEFeedForward 实例并设置为评估模式
x = torch.randn((4, 16, 512))
output = moe_ffn(x)
print(f'输出张量：shape = {output.shape}, 辅助损失：aux_loss = {moe_ffn.aux_loss}')

输出张量：shape = torch.Size([4, 16, 512]), 辅助损失：aux_loss = 0


In [26]:
del moe_ffn

之前提到过，MiniMind MoE 和 MiniMind Dense 的最大差别在于 FFN 模块的不同，我们可以对之前声明的 MiniMindBlock 进行继承，添加 MoE FFN 选项.

In [27]:
# 定义使用 MoE 的 MiniMind Block
class MiniMindBlock_DM(MiniMindBlock):
    def __init__(self, layer_id: int, config: LMConfig):
        super().__init__(layer_id, config)
        self.feed_forward = FeedForward(config) if not config.use_moe else MoEFeedForward(config)  # 根据配置选择前馈网络类型

In [28]:
# 检查继承是否正常
miniblock_dm = MiniMindBlock_DM(1, LMConfig_MoE)
print(f'类变量属性检查：layer_id = {miniblock_dm.layer_id}, 类函数属性检查：forward func = {miniblock_dm.forward}')

类变量属性检查：layer_id = 1, 类函数属性检查：forward func = <bound method MiniMindBlock.forward of MiniMindBlock_DM(
  (attention): Attention(
    (wq): Linear(in_features=512, out_features=512, bias=False)
    (wk): Linear(in_features=512, out_features=128, bias=False)
    (wv): Linear(in_features=512, out_features=128, bias=False)
    (wo): Linear(in_features=512, out_features=512, bias=False)
    (attn_dropout): Dropout(p=0.0, inplace=False)
    (resid_dropout): Dropout(p=0.0, inplace=False)
  )
  (attention_norm): RMSNorm()
  (ffn_norm): RMSNorm()
  (feed_forward): MoEFeedForward(
    (experts): ModuleList(
      (0-3): 4 x FeedForward(
        (w1): Linear(in_features=512, out_features=1408, bias=False)
        (w2): Linear(in_features=1408, out_features=512, bias=False)
        (w3): Linear(in_features=512, out_features=1408, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (gate): MoEGate()
    (shared_experts): FeedForward(
      (w1): Linear(in_features=512, out

我们仍然设置一个 batch size = 4, seq len =16, emb dim = 512 的输入向量，在 MoE 门控单元前向传播进行观察.

In [29]:
miniblock_dm.eval() # 冻结参数，直接 moe infer
x = torch.randn((4, 16, 512))
pos_cis = precompute_pos_cis(64, 16)
output, past_kv = miniblock_dm(x, pos_cis, use_cache=True)
print(f'输出张量：shape = {output.shape}, KV Cache: shape Key = {past_kv[0].shape}, shape Value = {past_kv[1].shape}')
print(f'辅助损失：aux_loss = {miniblock_dm.feed_forward.aux_loss}')

输出张量：shape = torch.Size([4, 16, 512]), KV Cache: shape Key = torch.Size([4, 16, 2, 64]), shape Value = torch.Size([4, 16, 2, 64])
辅助损失：aux_loss = 0


In [30]:
del miniblock_dm

如此，我们便完成了包含 MoE 和 Dense 两种 FFN 选项的 MiniMind Block 的定义，我们对此前声明的 MiniMindLM 作继承修改，使其具备 MoE 架构.

In [31]:
class MiniMindLM_DM(MiniMindLM):
    def __init__(self, params: LMConfig = None):
        super().__init__(params)
        structure = 'MoE' if params.use_moe else 'Dense'
        print(f'Initializing MiniMind {structure} Model...\n')
        self.layers = nn.ModuleList([MiniMindBlock_DM(l, params) for l in range(self.n_layers)])

In [32]:
# 检查一下
a = MiniMindLM_DM(LMConfig_Dense)
b = MiniMindLM_DM(LMConfig_MoE)
del a, b

Initializing MiniMind Dense Model...

Initializing MiniMind MoE Model...



接下来，我们设置一条长度为 4 的 token 序列，使用 MiniMindLM 对其执行一次前向传播，观察传播过程与返回值.

In [33]:
MiniMind_MoE = MiniMindLM_DM(LMConfig_MoE)
input_ids = torch.Tensor([1,  3,  5,  7]).long().reshape(1,  4)
OUT = MiniMind_MoE(input_ids,  use_cache=True)
print(f'返回 logits：size = {OUT.logits.shape}, 返回 aux_loss: {OUT.aux_loss},  返回 KV Cache List： len = {len(OUT.past_key_value)}')

Initializing MiniMind MoE Model...

====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

返回 logits：size = torch.Size([1, 4, 6400]), 返回 aux_loss: 0,  返回 KV Cache List： len = 2


In [34]:
# 我们让 MiniMind 根据我们设计的 四个输入 token 生成输出
out = MiniMind_MoE.generate(input_ids,  max_new_tokens=8,  use_cache=True)
print(f'生成结果：{out}')

gernerating new token: idx = 4
====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

gernerating new token: idx = 5
====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 1, 2, 64]), size_cache_v = torch.Size([1, 1, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 1, 2, 64]), size_cache_v = torch.Size([1, 1, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

gernerating new token: idx = 6
====> forward propagation started, num_minimind_blocks = 2
--------

In [35]:
del MiniMind_MoE

## 参考资料

- [十分钟读懂旋转编码（RoPE）](https://www.zhihu.com/tardis/zm/art/647109286?source_id=1003)
- [混合专家模型 MoE 的全面指南](https://blog.csdn.net/Code1994/article/details/145209710#:~:text=%E4%B8%BA%E4%BA%86%E5%9C%A8%E8%AE%AD%E7%BB%83%E8%BF%87%E7%A8%8B%E4%B8%AD%E5%AE%9E%E7%8E%B0%E4%B8%93%E5%AE%B6%E7%9A%84%E6%9B%B4%E5%9D%87%E5%8C%80%E5%88%86%E5%B8%83%EF%BC%8C%E8%BE%85%E5%8A%A9%E6%8D%9F%E5%A4%B1%EF%BC%88%E4%B9%9F%E7%A7%B0%E4%B8%BA%E8%B4%9F%E8%BD%BD%E5%9D%87%E8%A1%A1%E6%8D%9F%E5%A4%B1%EF%BC%89%E8%A2%AB%E6%B7%BB%E5%8A%A0%E5%88%B0%E4%BA%86%E7%BD%91%E7%BB%9C%E7%9A%84%E5%B8%B8%E8%A7%84%E6%8D%9F%E5%A4%B1%E4%B8%AD%E3%80%82%20%E5%AE%83%E5%A2%9E%E5%8A%A0%E4%BA%86%E4%B8%80%E4%B8%AA%E7%BA%A6%E6%9D%9F%EF%BC%8C%E8%BF%AB%E4%BD%BF%E4%B8%93%E5%AE%B6%E5%85%B7%E6%9C%89%E7%9B%B8%E7%AD%89%E7%9A%84%E9%87%8D%E8%A6%81%E6%80%A7%E3%80%82,%E8%BF%99%E4%B8%AA%E8%BE%85%E5%8A%A9%E6%8D%9F%E5%A4%B1%E7%9A%84%E7%AC%AC%E4%B8%80%E4%B8%AA%E7%BB%84%E6%88%90%E9%83%A8%E5%88%86%E6%98%AF%E5%9C%A8%E6%95%B4%E4%B8%AA%E6%89%B9%E6%AC%A1%E4%B8%AD%E5%AF%B9%E6%AF%8F%E4%B8%AA%E4%B8%93%E5%AE%B6%E7%9A%84%E8%B7%AF%E7%94%B1%E5%99%A8%E5%80%BC%E8%BF%9B%E8%A1%8C%E6%B1%82%E5%92%8C%EF%BC%9A%20%E8%BF%99%E4%B8%BA%E6%88%91%E4%BB%AC%E6%8F%90%E4%BE%9B%E4%BA%86%E6%AF%8F%E4%B8%AA%E4%B8%93%E5%AE%B6%E7%9A%84%E9%87%8D%E8%A6%81%E6%80%A7%E5%88%86%E6%95%B0%EF%BC%8C%E8%BF%99%E4%BA%9B%E5%88%86%E6%95%B0%E8%A1%A8%E7%A4%BA%E6%97%A0%E8%AE%BA%E8%BE%93%E5%85%A5%E6%98%AF%E4%BB%80%E4%B9%88%EF%BC%8C%E7%BB%99%E5%AE%9A%E4%B8%93%E5%AE%B6%E8%A2%AB%E9%80%89%E4%B8%AD%E7%9A%84%E5%8F%AF%E8%83%BD%E6%80%A7%E3%80%82%20%E6%88%91%E4%BB%AC%E5%8F%AF%E4%BB%A5%E5%88%A9%E7%94%A8%E8%BF%99%E4%BA%9B%E5%88%86%E6%95%B0%E6%9D%A5%E8%AE%A1%E7%AE%97%E5%8F%98%E5%BC%82%E7%B3%BB%E6%95%B0%EF%BC%88CV%EF%BC%89%EF%BC%8C%E5%AE%83%E5%91%8A%E8%AF%89%E6%88%91%E4%BB%AC%E4%B8%93%E5%AE%B6%E4%B9%8B%E9%97%B4%E7%9A%84%E9%87%8D%E8%A6%81%E6%80%A7%E5%88%86%E6%95%B0%E7%9A%84%E5%B7%AE%E5%BC%82%E7%A8%8B%E5%BA%A6%E3%80%82)